In [1]:
import numpy as np
from numpy import array as ar
import matplotlib.pyplot as plt
from glob import glob
from skimage.io import imread
from multiprocessing import Pool, cpu_count
%matplotlib inline
%load_ext line_profiler

In [2]:
import pyqtgraph as pq
%gui qt

In [3]:
paths = ['/groups/ahrens/ahrenslab/davis/data/epi/20170621/6dpf_cy221xcy221_f1_spon_1/Pos0/',
        '/groups/ahrens/ahrenslab/davis/data/epi/20170621/6dpf_cy221xcy221_f1_spon_2/Pos0/'
        '/groups/ahrens/ahrenslab/davis/data/epi/20170621/6dpf_cy221xcy221_f2_spon_1/Pos0/',
        '/groups/ahrens/ahrenslab/davis/data/epi/20170621/6dpf_cy221xcy221_f2_spon_2/Pos0/',
        '/groups/ahrens/ahrenslab/davis/data/epi/20170621/6dpf_cy221xcy221_f5_spon_1_1/Pos0/']

In [4]:
im_fnames = [glob(d + '*.tif') for d in paths]
[imf.sort() for imf in im_fnames]
print(len(im_fnames))

4


In [5]:
def wipe_ims(pq):
    from numpy import zeros
    for iw in pq.images:
      iw.setImage(zeros((1,1))) #<- dummy 1x1 image
      iw.parent().close()

def mem_use():
    import os
    import psutil
    pid = os.getpid()
    py = psutil.Process(pid)
    memory_use = py.memory_info()[0]/2.**30  # memory use in GB...I think
    return memory_use

def pool_wrapper(function, data, num_cores, mode='map'):
    from multiprocessing import Pool
    with Pool(num_cores) as p:
        try:
            if mode == 'map':
                result = p.map(function, data)
            elif mode == 'starmap':
                result = p.starmap(function, data)
        except Exception as inst:
            print('There was a problem!')
            print(inst)
            return None
    return result

In [6]:
%%time
multicore = True
to_load = -1
plr = slice(0,None)
num_cores = 16

if multicore:
    ims = np.array(pool_wrapper(imread, im_fnames[to_load][plr], num_cores))
else:
    ims = ar([imread(imf) for imf in im_fnames[to_load][plr]])

CPU times: user 9.5 s, sys: 23.4 s, total: 32.9 s
Wall time: 44.7 s


In [7]:
mem_use()

31.553890228271484

In [ ]:
ims.shape

In [41]:
# get a mask for the fish
def get_fish_mask(im, sigma=4, shrink_factor=.1, min_size=400, disk_size=14, return_sparse=True):
    from numpy import abs
    im_ = im.copy().astype('float32')
    # remove background
    im_bgr = remove_background(im, sigma=sigma)
    # estimate a threshold on the background-removed image
    thr = simple_threshold(im_bgr, shrink_factor=shrink_factor)
    # remove speckles of the absolute value of the background-subtracted image    
    binarized = despeckle_morph(abs(im_bgr) > thr, min_size=min_size)
    binarized_2 = binarized.copy()
    
    if binarized.any():
        # expand the mask
        dilated = disk_dilate(binarized, disk_size=disk_size)
        # threshold in the original image
        thr_2 = simple_threshold(im_[dilated], nbins=300)
        # remove speckles
        binarized_2 = despeckle_morph((im_ * dilated) > thr_2, min_size=min_size)      
    
    if return_sparse:
        from scipy.sparse import coo_matrix
        binarized_2 = coo_matrix(binarized_2)
        
    return binarized_2

def simple_threshold(data, shrink_factor=.1, nbins=3000):
    from numpy import histogram, argmax, where, median, argmin
    counts, bins = histogram(data.ravel(), bins=nbins)
    pk = median(data)
    pk_ind = argmin(abs(bins - pk))
    pk_counts = counts[pk_ind-1]
    right_tail = where(counts[pk_ind:] < (pk_counts * shrink_factor))[0]
    if len(right_tail) > 0:
        right_foot = right_tail[0] + pk_ind 
    else:
        right_foot = -1
    return bins[right_foot]

def remove_background(im, sigma=4):
    from scipy.ndimage.filters import gaussian_filter
    im_ = im.copy().astype('float32')
    return im_ - gaussian_filter(im_, sigma=sigma)

def despeckle_morph(im, min_size=32):
    from skimage.morphology import remove_small_objects
    return remove_small_objects(im, min_size=min_size)

# wrap binary dilation
def disk_dilate(im, disk_size=8):
    from skimage.morphology import disk, binary_dilation    
    # execution time of binary dilation depends on size of the image so we can speed things up by cropping the image            
    rmin, rmax, cmin, cmax = get_bbox(im, pad=disk_size//2)        
    result = im.copy()
    result[rmin:rmax, cmin:cmax] = binary_dilation(im[rmin:rmax, cmin:cmax], disk(disk_size))
    return result

# get the bounding box of a binary array with some padding
def get_bbox(img, pad=0):
    from numpy import any, where
    rows = any(img, axis=1)
    cols = any(img, axis=0)
    rmin, rmax = where(rows)[0][[0, -1]]
    cmin, cmax = where(cols)[0][[0, -1]]

    return rmin - pad, rmax + pad, cmin - pad, cmax + pad
    

# Remove the tail, leaving the brain
# todo: make this function loop until a single object meeting a size criterion is found
def get_brain(im, selem_size=6, object_size=200, return_sparse=True):
    import warnings
    from skimage.morphology import binary_opening, disk, remove_small_objects
    from scipy.sparse import issparse
    from numpy import array, zeros
    
    im_ = im.copy().astype('bool')
    brain = zeros(im_.shape, dtype='bool')
    
    # if the input is sparse, make it dense
    if issparse(im):
        im_ = array(im_.todense())
    
    if not im_.any():        
        if return_sparse:
            from scipy.sparse import coo_matrix
            brain = coo_matrix(brain)        
        return brain
    
    rmin, rmax, cmin, cmax = get_bbox(im_, pad=selem_size//2)
    
    # remove_small_objects raises a warning if you give it a boolean array with one object     
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")    
        im_[rmin:rmax, cmin:cmax] = binary_opening(im_[rmin:rmax, cmin:cmax], selem=disk(selem_size))
        brain = remove_small_objects(im_, min_size=object_size)
    
    if return_sparse:
        from scipy.sparse import coo_matrix
        brain = coo_matrix(brain)
        
    return brain

def cart2pol(x,y):
    from numpy import hypot, arctan2
    r = hypot(x,y)
    phi = arctan2(y, x)
    return r, phi

def rotmat2d(phi):
    from numpy import cos, sin, zeros
    mat = zeros([2,2])
    mat[0,0] = cos(phi)
    mat[1,1] = cos(phi)
    mat[0,1] = sin(phi)
    mat[1,0] = -sin(phi)    
    return mat

def angle_difference(x,y):
    from numpy import arctan2, sin, cos
    return arctan2(sin(x-y), cos(x-y))

def eig_wrapper(im):
    from numpy import where, cov, argsort
    import scipy.linalg as la
            
    y,x = np.where(im)
    evals, evecs = la.eig(cov(x,y))
    idx = argsort(evals)[::-1]
    evals = evals[idx]
    evecs = evecs[:,idx]
    
    return evals, evecs

def affine_wrapper(image, rotation, c_in, c_out, order=1):
    """
    Perform rotation followed by translation
    """
    from scipy.ndimage.interpolation import affine_transform
    offset = c_in - c_out.dot(rotation)
    return affine_transform(image, rotation.T, offset=offset, mode='wrap', order=order)


# todo: rotate a bounding box instead of the whole image
def orient_tail(im, return_sparse=True):
    # point the tail of the fish toward the origin, parallel to the x-axis
    # im must be a binarized fish mask
    from numpy import pi, rad2deg, isnan, array, zeros, round, roll, abs, argmin
    from skimage.transform import rotate
    from scipy.ndimage.measurements import center_of_mass
    from scipy.ndimage.interpolation import shift
    from scipy.sparse import issparse, coo_matrix

    im_ = im.copy()
    
    if issparse(im_):
        im_ = array(im_.todense())
    
    phi = 0
    dydx = 0
    
    # if we can't find a fish or a brain, return all 0s
    if not im_.any():
        return coo_matrix(zeros(im_.shape, dtype='bool')), phi, dydx
    
    brain = get_brain(im_, return_sparse=False)
    
    # if we can't find a fish or a brain, return all 0s
    if not brain.any():
        output = zeros(im_.shape, dtype='bool')
        if return_sparse:
            output = coo_matrix(output)
        return output, phi, dydx
    
    raw_com = array(center_of_mass(im_))
    brain_com = array(center_of_mass(brain))    
           
    y_, x_ = brain_com
    tail_y, tail_x = brain_com - raw_com
    
    brain_tail_angle = cart2pol(tail_x, tail_y)[-1]    
        
    # get the orietation of the long axis of the brain        
    evals, evecs = eig_wrapper(brain)    
    brain_angle = cart2pol(evecs[0,0], evecs[1,0])[-1]
    candidate_phis = array([brain_angle, brain_angle + pi])
    # Eigenvector of brain is ambiguous between tail to the left and tail to the right. Pick angle that 
    # minimizes angular distance from brain-tail angle
    which_angle = argmin([abs(angle_difference(brain_tail_angle, candidate)) for candidate in candidate_phis])
    #which_angle = argmin(abs(candidate_phis - brain_tail_angle) % (2 * np.pi))
    phi = candidate_phis[which_angle]
    
    # todo: perform rotation on a cropped region of the image
    result = rotate(im_, angle=rad2deg(phi), center = (x_, y_))
    
    # shift resulting image to center of field of view
    dydx = (result.shape[0]/2)  - y_, (result.shape[1]/2) - x_
    # for performance, avoid sub-pixel shift
    result_shifted = roll(result, round(dydx).astype('int'), axis=(0,1))
    
    if return_sparse:
        result_shifted = coo_matrix(result_shifted)
    
    return result_shifted, phi, dydx


def get_cropped_fish(image, phi, dydx, crop_window):
    # given an image, a rotation angle, and a shift, apply the shift, then the rotation, then crop around the
    # center of the transformed image and apply a binary mask, returning an image of the fish nervous system on a
    # a background of 0s    
    from scipy.ndimage import shift
    from skimage.transform import rotate
    from numpy import roll, round
    #shifted = shift(image, dydx, mode='wrap')
    shifted = roll(image, round(dydx).astype('int'), axis=(0,1))
    rotated = rotate(shifted, angle=np.rad2deg(phi), mode='wrap', preserve_range=True)    
    mid = np.array(rotated.shape) // 2 
    
    window_y, window_x = crop_window

    # we take the transpose to cancel the effect of this kind of indexing
    crop = rotated[mid[0] + window_y, mid[1] + window_x].T
    mask = get_fish_mask(crop, return_sparse=False)
    
    return (crop * mask).astype('int16')

def align_brains(static, moving):
    from dipy.align.imaffine import AffineRegistration, AffineMap
    from dipy.align.transforms import RigidTransform2D 
    from scipy.ndimage.measurements import center_of_mass
    from scipy.ndimage import affine_transform
    from numpy import round, eye, array
          
    affreg = AffineRegistration(verbosity=0)    
    rigid = RigidTransform2D()
            
    static_, moving_ = static.copy(), moving.copy()
    coms = (0,0)   
    
    static_brain = get_brain(static_, return_sparse=False)
    moving_brain = get_brain(moving_, return_sparse=False)
    
    fill_value = static[static_brain].min()
    
    if not moving_brain.any():
        return moving_, coms, AffineMap(eye(3))
    
    # isolate region of interest for registration
    coms = array(round(center_of_mass(static_brain)).astype('int'))
    static_brain_cropped = (static_brain * static)[coms[0] - 20:coms[0] + 20, coms[1] - 30:coms[1] + 30]
    moving_brain_cropped = (moving_brain * moving)[coms[0] - 20:coms[0] + 20, coms[1] - 30:coms[1] + 30]
        
    # set background values to minimum of brain to mitigate effect of masking on registration
    static_brain_cropped[static_brain_cropped==0] = static_[static_brain].min()
    moving_brain_cropped[moving_brain_cropped==0] = moving_[moving_brain].min()
    
    # estimate rigid transformation between static and moving
    g2w = eye(3)
    g2w[:2,-1] = -array(static_brain_cropped.shape) / 2
    
    params0 = None
    starting_affine = None
    tx = affreg.optimize(static_brain_cropped, moving_brain_cropped, rigid, params0, static_grid2world=g2w, moving_grid2world=g2w, starting_affine=starting_affine)
    
    dydx = tx.affine[:2,-1]
    moving_[moving_ == moving_.min()] = fill_value
    tx.domain_grid2world[:2,-1] = -coms
    warped = tx.transform(moving_, sampling_grid_shape=(moving_.shape))
    return warped, coms, tx

def get_tail_angle(images, center, radius):
    from skimage.draw import circle_perimeter    
    from numpy import arctan2, argsort, pi, argmax, unwrap, arange
    
    cent_y, cent_x = center
    
    x_c, y_c = circle_perimeter(cent_x, cent_y, radius=radius, shape = images[-1].shape)
    angles = arctan2(y_c - cent_y, x_c - cent_x)
    inds = np.argsort(angles)
    #keep = inds[arange(-25,25, dtype='int')]
    keep = inds
    x_tail = x_c[keep]
    y_tail = y_c[keep]
    tail_arc = images[:, y_tail, x_tail]
    tail_angle = argmax(tail_arc, axis=1)
    tail_angle = unwrap(angles[keep][tail_angle]) + pi
    tail_angle = tail_angle.astype('float32')
    
    return tail_angle, (x_tail, y_tail), angles

In [42]:
# area of the brain in pixels
min_brain_size = 750

## Get fish masks

In [10]:
%%time
fish_masks = np.array(pool_wrapper(get_fish_mask, ims, num_cores))

CPU times: user 13.3 s, sys: 15.2 s, total: 28.4 s
Wall time: 1min 55s


In [11]:
fmasks = np.array([fm.todense() for fm in fish_masks])
pq.image(fmasks)

## Check brain segmentation

In [43]:
%%time
brain_masks = np.array(pool_wrapper(get_brain, fish_masks, num_cores))

CPU times: user 2.81 s, sys: 5.21 s, total: 8.02 s
Wall time: 25.3 s


In [17]:
bmasks = np.array([bm.todense() for bm in brain_masks])
pq.image(bmasks)

## Get oriented masks and coarse transform parameters

In [44]:
%%time
oriented_masks, rotations, translations = zip(*pool_wrapper(orient_tail, fish_masks, num_cores))

CPU times: user 3.35 s, sys: 5.42 s, total: 8.77 s
Wall time: 1min 4s


In [45]:
mem_use()

48.285804748535156

Use transform parameters to generate cropped, masked fish image

In [46]:
%%time
# the crop window is relative to the center of mass of the brain
window_x = np.arange(-200,40, dtype='int').reshape(-1,1)
window_y = np.arange(-100,100)

crop_window = [(window_y, window_x)] * len(ims)
oriented_ims = np.array(pool_wrapper(get_cropped_fish, zip(ims, rotations, translations, crop_window), num_cores, mode='starmap'))

CPU times: user 14.7 s, sys: 18.9 s, total: 33.6 s
Wall time: 1min 35s


In [47]:
mem_use()

48.31112289428711

In [ ]:
pq.image(oriented_ims)

## Do fine alignment on oriented fish images

In [48]:
%%time
static = [oriented_ims[-1]] * oriented_ims.shape[0]
aligned_ims, coms, txs = zip(*pool_wrapper(align_brains, zip(static, oriented_ims), num_cores=num_cores, mode='starmap'))

CPU times: user 9.5 s, sys: 20.4 s, total: 29.9 s
Wall time: 4min 29s


In [49]:
pqim = pq.image(np.array(aligned_ims))

In [ ]:
pqim.roi.getArrayRegion(pqim.image, pqim.imageItem, axes=((1,2)), returnMappedCoords=True)[1]

In [ ]:
wipe_ims(pq)

In [ ]:
# this roi works for '/groups/ahrens/ahrenslab/davis/data/epi/20170621/6dpf_cy221xcy221_f2_spon_1/Pos0/'
from scipy.ndimage.filters import median_filter
roi = (slice(0,None), slice(94,99), slice(190,194))
fs_im = 100
t = np.arange(oriented_ims.shape[0])[roi[0]] / fs_im
fig, axs = plt.subplots(nrows=2, figsize=(12,8))
ax_angle = axs[0]
ax_angle.plot(t, get_tail_angle(oriented_ims[roi[0]], (100,200), 90)[0], color='grey')
ax_angle.set_ylabel('Radians')
ts = oriented_ims[roi].mean((1,2))
floor_ = 1800
ceil_ = None
ax_fluo = ax_angle.twinx()
#ax_fluo.plot(t, ts.clip(floor_, ceil_), alpha = .4)
ax_fluo.plot(t, median_filter(ts, size=200).clip(floor_, ceil_), linewidth=2, color='k', alpha=.7)
ax_fluo.set_ylabel('Fluorescence Intensity [au]')
axs[0].legend([ax_angle.lines[-1], ax_fluo.lines[-1]], ['Tail angle','Smoothed fluorescence'], )

axs[0].set_xlabel('Time [s]')

from matplotlib.patches import Rectangle
axs[1].imshow(oriented_ims[roi[0]].mean(0), cmap='gray', origin='lower')
rect = Rectangle((roi[2].start, roi[1].start), roi[2].stop - roi[2].start, roi[1].stop - roi[1].start, fill=False, color='r')
axs[1].add_patch(rect)

# Test quality of tail orientation 

In [ ]:
t = 21486
from scipy.ndimage.measurements import center_of_mass
raw_brain = np.array(brain_masks[t].todense())
raw_fish = np.array(fish_masks[t].todense())

aligned_fish = orient_tail(raw_fish)[0]

com_brain = center_of_mass(raw_brain)
com_fish = center_of_mass(raw_fish)

fig, axs = plt.subplots(ncols=2, figsize=(16,9))
axs[0].imshow(raw_brain.astype('int') + raw_fish.astype('int'), origin='lower')
axs[0].plot(*com_brain[::-1],'o')
axs[0].plot(*com_fish[::-1],'o')

axs[1].imshow(aligned_fish.todense(), origin='lower')
#axs[0].plot(*com_brain[::-1],'o')
#axs[0].plot(*com_fish[::-1],'o')

In [ ]:
t = -5000
test_im = ims[plr[t]]

In [ ]:
fig, axs = plt.subplots(ncols=3, figsize=(12,4))
from scipy.ndimage import shift
from skimage.transform import rotate
phi = rotations[t]
dydx= translations[t]
shifted = shift(test_im, dydx, mode='wrap')
rotated = rotate(shifted, angle=np.rad2deg(phi), mode='wrap')

axs[0].imshow(test_im, cmap='hsv', origin='lower')
axs[1].imshow(shifted, cmap='hsv', origin='lower')
axs[2].imshow(rotated, cmap='hsv', origin='lower')
axs[2].set_xlim(np.array(rotated.shape) / 2 + [-200,40] )
axs[2].set_ylim(np.array(rotated.shape) / 2 + [-100,100] )

In [ ]:
t = -5000
plt.imshow(get_cropped_fish(ims[plr[t]], oriented[t][1], oriented[t][-1]), origin='lower')

In [ ]:
pq.image(oriented_ims)

In [ ]:
fig, axs = plt.subplots(figsize=(9,3))
axs.title.set_text('Brain pixels over time')
axs.plot(brains.sum((1,2)), color='k')

In [ ]:
pq.image(brains)

In [ ]:
pq.image(processed)

In [ ]:
%%time
ref = processed[-1]
p = Pool(num_cores)
tmp = ar(p.map(brain_reg,  zip([ref] * processed.shape[0], processed)));
p.close()
p.join()

In [ ]:
aligned_profiles = ar([t[0] for t in tmp])

In [ ]:
pq.image(aligned_profiles)

In [ ]:
from skimage.transform import rotate
from scipy.ndimage.measurements import center_of_mass
moving = rotate(processed[9105], 45)

raw_com = np.array(center_of_mass(moving))
brain_com = np.array(center_of_mass(get_brain(moving)))    
y_, x_ = brain_com

fig, axs = plt.subplots(ncols = 2, figsize=(12,4))
axs[0].imshow(moving, origin='lower', cmap='gray_r')

tail_y, tail_x = raw_com - brain_com
brain_tail_angle = np.pi + cart2pol(tail_x, tail_y)[-1]
axs[0].arrow(x_, y_, 3 * tail_x, 3 * tail_y, color='y')

evals, evecs = eig_wrapper(get_brain(moving, 4, 400))
brain_angle = cart2pol(evecs[0,0], evecs[1,0])[-1]

axs[0].arrow(x_, y_, evecs[0][0] * np.real(evals[0]), evecs[1][0] * np.real(evals[0]), color='r')
axs[0].arrow(x_, y_, evecs[0][1] * np.real(evals[1]), evecs[1][1] * np.real(evals[1]), color='m')
axs[0].arrow(x_, y_, tail_x, tail_y)

#brain_angle = min(brain_angle, (np.pi + brain_angle) % 2 * np.pi)
phi = brain_angle + (brain_angle - brain_tail_angle)
phi = min(phi, (np.pi + phi) % 2*np.pi)

axs[1].imshow(orient_tail(moving)[0], cmap='gray_r', origin='lower')
#axs[1].imshow(rotate(moving, angle=np.rad2deg(phi), center=(x_,y_)), origin='lower', cmap='gray_r')

#lims = (x_ - 100, x_ + 100), (y_ - 100, y_ + 100)
#[ax.set_xlim(*lims[0]) for ax in axs]
#[ax.set_ylim(*lims[1]) for ax in axs]

## Registration debugging 

In [ ]:
from skimage.io import imread
from skimage import data_dir
from skimage.transform import rotate
from numpy import rad2deg
from dipy.align.imaffine import AffineRegistration
from dipy.align.transforms import RotationTransform2D 

affreg = AffineRegistration(verbosity=0)
rotation = RotationTransform2D()

static = imread(data_dir + "/phantom.png", as_grey=True)

rot_phi = .3
moving = rotate(static, rad2deg(rot_phi))
mg2w = np.array([[1,0,-moving.shape[0]//2],[0,1,-moving.shape[1]//2],[0,0,1]])
sg2w = mg2w
params0 = None
starting_affine = None
tx = affreg.optimize(static, moving, rotation, params0, static_grid2world=sg2w, moving_grid2world=mg2w, starting_affine=starting_affine)
titles = ['Static', 'Moving', 'Transformed Moving']

samp_g2w = np.eye(3)
samp_g2w[0,-1] = -200
samp_g2w[1,-1] = -200

fig, axs = plt.subplots(ncols = 3)
axs[0].imshow(static, origin='lower')
axs[1].imshow(moving, origin='lower')
axs[2].imshow(tx.transform(moving), origin='lower')
[axs[ind].title.set_text(val) for ind, val in enumerate(titles)]

print('True angle      : {0}'.format(rot_phi))
print('Estimated angle : {0}'.format(np.arccos(tx.affine[0,0])))

In [ ]:
from dipy.align.imaffine import AffineMap
from skimage.transform import AffineTransform
from skimage.transform import warp

In [ ]:
g2ws = np.eye(3)
g2ws[0,2] = -(static.shape[0] / 2)
g2ws[1,2] = -(static.shape[1] / 2)

g2wi = np.eye(3)
g2wi[0,2] = -(static.shape[0] / 2) - 100
g2wi[1,2] = -(static.shape[0] / 2)

affmap_r = AffineMap(np.eye(3))
affmap_r.codomain_shape = static.shape
affmap_r.domain_shape = static.shape

phi = np.pi
affmap_r.affine[:2,:2] = rotmat2d(phi)

dy, dx = 0,-100
affmap_r.affine[0,-1] = dy
affmap_r.affine[1,-1] = dx
rotated = affmap_r.transform(static, sampling_grid_shape=static.shape, sampling_grid2world=g2ws, image_grid2world=g2wi)

affmap_t = AffineMap(np.eye(3))
affmap_t.codomain_shape = static.shape
affmap_t.domain_shape = static.shape

dx, dy = 0,0
affmap_t.affine[0,-1] = dx
affmap_t.affine[1,-1] = dy

translated = affmap_t.transform(rotated, sampling_grid_shape=static.shape, sampling_grid2world = g2ws, image_grid2world=g2wi)
fig, axs = plt.subplots(ncols=2)
axs[0].imshow(static + rotated, origin='lower')
axs[1].imshow(static + translated, origin='lower')

In [ ]:
plt.imshow(affine_wrapper(static, affmap.affine[:2,:2], np.array(static.shape) / 2, np.array(static.shape) / 2 + (0,0)), origin='lower')

In [ ]:
transform = affmap.affine[:2,:2]
c_in = np.array(static.shape) / 2
c_out = np.array(static.shape) / 2 + (100,100)
offset = c_in - c_out.dot(transform)
plt.imshow(affine_transform(static, transform.T, offset=offset), origin='lower')

In [ ]:
from skimage import data
from skimage import transform
import numpy as np
import matplotlib.pyplot as plt

image = data.chelsea()[:,:,0]
def rotate_and_center(image, rotation_center, phi):
    from numpy import array
    from skimage.transform import warp, AffineTransform
    origin_y, origin_x = rotation_center
    shift_y, shift_x = array(image.shape) / 2.
    tf_rotate = AffineTransform(rotation=phi)
    tf_shift = AffineTransform(translation=[-origin_x, -origin_y])
    tf_shift_inv = AffineTransform(translation=[shift_x, shift_y])

    image_rotated = transform.warp(image, (tf_shift + (tf_rotate + tf_shift_inv)).inverse)
    return image_rotated

fig, axs = plt.subplots(ncols=2)
axs[0].imshow(image, origin='lower')
axs[1].imshow(rotate_and_center(image, (100,200), np.pi), origin='lower')